In [1]:
!pip install praw
!pip install nltk
!pip install sqlalchemy
!pip install psycopg2-binary

In [2]:
from configparser import ConfigParser
import praw
from praw.models import MoreComments
import pandas as pd
import matplotlib.pyplot as plt
import json
import plotly
import plotly.express as px
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import requests
from flask import Flask, render_template
from datetime import datetime, timedelta 
from sqlalchemy import create_engine


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rudy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rudy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rudy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Rudy\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Rudy\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:


reddit = praw.Reddit(
      client_id= '',
      client_secret = '',
      password = '',
      user_agent = '',
      username = '',
      check_for_async=False
  )

  #coinmarketcap API call - limit to top 150 coins
url='https://web-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
for start in range(1, 100, 1000):
    params = {
        'start': start,
        'limit': 150,
            }
    r = requests.get(url, params=params)
    data = r.json()

ticker = [(crypto['symbol'].lower()) for crypto in (data['data'])]
slug = [(crypto['name'].lower()) for crypto in (data['data'])]

  #subreddit to set parameters for title, author, body, score
def reddit_scraper(reddit, subreddit_page, limit):
  df = pd.DataFrame()
  subreddit = reddit.subreddit(subreddit_page)
  hot_topic = subreddit.hot(limit=limit)

  for topics in hot_topic: 
    topic_list = topics.comments.list()
    for comments in topic_list:
      if isinstance(comments, MoreComments):
        continue
      df = df.append({'title': topics.title,
                      'author': comments.author,
                      'id': comments.id,
                      'comments': comments.body,
                      'created_utc': comments.created_utc,
                      'upvote_score': comments.score
                      },                                          
                      ignore_index=True)


  return df

wall = reddit_scraper(reddit, 'CryptoCurrency', 49)

tokenizer = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

#text cleaning; tokenization, stop words, lemmatization
def text_cleaner(df):
    cleaned_text = []
    for text in df:
        tokenize_words = tokenizer.tokenize(text)      
        stop_word_filter = [token.lower() for token in tokenize_words if token.lower() not in stop_words and not token.isdigit()]
        lemmitize_words = ([lemmatizer.lemmatize(token) for token in stop_word_filter])
        cleaned_text.extend(lemmitize_words)
    
    return cleaned_text

                    
def sentiment_analysis(df):
  return text_cleaner(df)

#convert utc time to datetime
def get_date(submission):
	time = submission.created
	return datetime.datetime.fromtimestamp(time)

#login to postgres
def postgres_login(user, password, host, port, db, table_name):   
    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')
    wall.to_sql(name=table_name, con=engine, if_exists='replace')
    
    
sia = SIA()

ps = []
for comment in wall.comments:
    #polarity scores for SentimentIntensityAnalyzer
    score = sia.polarity_scores(comment)
    score['replies'] = comment
    ps.append(score)

pol_df = pd.DataFrame.from_records(ps)

#set bullish if 1, bearish if 0, neutral if 0
pol_df['sentiment'] = 0
pol_df.loc[pol_df['compound'] > 0.2, 'sentiment'] = 1
pol_df.loc[pol_df['compound'] < -0.2, 'sentiment'] = -1


bullish = sentiment_analysis(pol_df[pol_df.sentiment==1].replies)
bearish = sentiment_analysis(pol_df[pol_df.sentiment==-1].replies)
neutral = sentiment_analysis(pol_df[pol_df.sentiment==0].replies)

#bullish ticker & names 
df_bull_ticker = pd.DataFrame(nltk.FreqDist([crypto for bull in text_cleaner(bullish) for crypto in ticker if bull==crypto]).most_common(5), columns=['Name', 'Frequencies'])
df_bull_slug = pd.DataFrame(nltk.FreqDist([crypto for bull in text_cleaner(bullish) for crypto in slug if bull==crypto]).most_common(5), columns=['Name', 'Frequencies'])
#bearish ticker & names
df_bear_ticker = pd.DataFrame(nltk.FreqDist([crypto for bear in text_cleaner(bearish) for crypto in ticker if bear==crypto]).most_common(5), columns=['Name', 'Frequencies'])
df_bear_slug = pd.DataFrame(nltk.FreqDist([crypto for bear in text_cleaner(bearish) for crypto in slug if bear==crypto]).most_common(5), columns=['Name', 'Frequencies'])
#neutral ticker & names
df_neutral_ticker = pd.DataFrame(nltk.FreqDist([crypto for neu in text_cleaner(neutral) for crypto in ticker if neu==crypto]).most_common(5), columns=['Name', 'Frequencies'])
df_neutral_slug = pd.DataFrame(nltk.FreqDist([crypto for neu in text_cleaner(neutral) for crypto in slug if neu==crypto]).most_common(5), columns=['Name', 'Frequencies'])


C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipy

C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipy

C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipy

C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipy

C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipy

...

C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipy

C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipy

C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipy

C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipykernel_19324\1298406959.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': topics.title,
C:\Users\Rudy\AppData\Local\Temp\ipy

'  \n    bulltickerfig = px.bar(df_bull_ticker, x=df_bull_ticker.Name, y=df_bull_ticker.Frequencies)\n    bullslugfig = px.bar(df_bull_slug, x=df_bull_slug.Name, y=df_bull_slug.Frequencies)\n    bulltickergraphJSON = json.dumps(bulltickerfig, cls=plotly.utils.PlotlyJSONEncoder)\n    bullsluggraphJSON = json.dumps(bullslugfig, cls=plotly.utils.PlotlyJSONEncoder)\n\n    beartickerfig = px.bar(df_bear_ticker, x=df_bear_ticker.Name, y=df_bear_ticker.Frequencies)\n    bearslugfig = px.bar(df_bear_slug, x=df_bear_slug.Name, y=df_bear_slug.Frequencies)\n    beartickergraphJSON = json.dumps(beartickerfig, cls=plotly.utils.PlotlyJSONEncoder)\n    bearsluggraphJSON = json.dumps(bearslugfig, cls=plotly.utils.PlotlyJSONEncoder)\n\n    neutickerfig = px.bar(df_neutral_ticker, x=df_neutral_ticker.Name, y=df_neutral_ticker.Frequencies)\n    neuslugfig = px.bar(df_neutral_slug, x=df_neutral_slug.Name, y=df_neutral_slug.Frequencies)\n    neutickergraphJSON = json.dumps(neutickerfig, cls=plotly.utils.Pl

In [4]:
wall['created_utc'] = wall['created_utc'].apply(lambda x: datetime.fromtimestamp(x))

In [5]:
df_bull = pd.concat([df_bull_ticker, df_bull_slug], axis=1, ignore_index=True)
df_bear = pd.concat([df_bear_ticker, df_bear_slug], axis=1, ignore_index=True)
df_neutral = pd.concat([df_neutral_ticker, df_neutral_slug], axis=1, ignore_index=True)
df_all = pd.concat([df_bull, df_bear, df_neutral], axis=1, ignore_index=True)


In [6]:
df_all.columns = ['Bull Ticker', 'Bull_T Freq','Bull Slug','Bull_S Freq','Bear Ticker', 'Bear_T Freq','Bear Slug','Bear_S Freq','Neutral Ticker', 'Neutral_T Freq','Neutral Slug','Neutral_S Freq']


In [7]:
df_all = df_all.join(wall['created_utc'])

In [8]:
df_all.set_index('created_utc', inplace=True)
wall.set_index('created_utc', inplace=True)

In [9]:
wall.head()

,title,author,id,comments,upvote_score
created_utc,,,,,
2023-01-29 00:26:15,MoonPlace - a place like r/place but on-chain ...,Dismal-Emu-3855,j6burpt,For those people not sure how to upload more c...,23
2023-01-29 11:42:47,MoonPlace - a place like r/place but on-chain ...,AutisticGayBear69,j6dng9d,I am wondering if it might be a good idea to a...,18
2023-01-29 03:42:16,MoonPlace - a place like r/place but on-chain ...,SomeRandomTito,j6cawtw,This should be the banner of r/CryptoCurrency....,9
2023-01-29 05:01:35,MoonPlace - a place like r/place but on-chain ...,randomFrenchDeadbeat,j6cgo7d,Wow. Everything is sold XD\n\nThis is the firs...,8
2023-01-28 19:32:19,MoonPlace - a place like r/place but on-chain ...,Lord-Nagafen,j6aux29,Added Batman to the canvas! 🦇,7


In [10]:
df_all.head()

,Bull Ticker,Bull_T Freq,Bull Slug,Bull_S Freq,Bear Ticker,Bear_T Freq,Bear Slug,Bear_S Freq,Neutral Ticker,Neutral_T Freq,Neutral Slug,Neutral_S Freq
created_utc,,,,,,,,,,,,
2023-01-29 00:26:15,one,190,bitcoin,109,one,80,bitcoin,51,one,106,bitcoin,43
2023-01-29 11:42:47,btc,97,chain,23,btc,34,chain,9,btc,72,monero,12
2023-01-29 03:42:16,eth,53,cardano,19,eth,28,cardano,8,eth,44,chain,7
2023-01-29 05:01:35,op,40,cosmos,17,elon,15,monero,5,matic,19,polygon,6
2023-01-28 19:32:19,elon,25,dogecoin,13,etc,12,ethereum,4,elon,19,cardano,6


In [11]:
wall.dtypes

title           object
author          object
id              object
comments        object
upvote_score     int64
dtype: object

In [18]:
wall.title = wall.title.astype("string")
wall.author = wall.author.astype("string")
wall.id = wall.id.astype("string")
wall.comments = wall.comments.astype("string")

In [20]:
postgres_login('','','',6969,'reddit','reddit_r_Cryptocurrency')

In [33]:
query = """SELECT comments, author
FROM public."reddit_r_Cryptocurrency"
WHERE upvote_score >10"""

pd.read_sql(query, con=engine)

,comments,author
0,For those people not sure how to upload more c...,Dismal-Emu-3855
1,I am wondering if it might be a good idea to a...,AutisticGayBear69
2,Working on upgrading server at the moment...,jwinterm
3,Can definitely see avatar enthusiasts minting ...,nanooverbtc
4,300k moons creating a massive pixel art cock n...,IHaventEvenGotADog
...,...,...
273,When so many people are so sure that we alread...,excalilbug
274,Wanna know a secret? The bond market. It will ...,StarkillerZ69
275,Bond. James Bond.,Zeric79
276,>the early holder exchanged 4.5 million tokens...,MaeronTargaryen


In [32]:
query = """SELECT *
FROM public."reddit_r_Cryptocurrency"
LIMIT 10"""

pd.read_sql(query, con=engine)

,created_utc,title,author,id,comments,upvote_score
0,2023-01-29 00:26:15,MoonPlace - a place like r/place but on-chain ...,Dismal-Emu-3855,j6burpt,For those people not sure how to upload more c...,23
1,2023-01-29 11:42:47,MoonPlace - a place like r/place but on-chain ...,AutisticGayBear69,j6dng9d,I am wondering if it might be a good idea to a...,18
2,2023-01-29 03:42:16,MoonPlace - a place like r/place but on-chain ...,SomeRandomTito,j6cawtw,This should be the banner of r/CryptoCurrency....,9
3,2023-01-29 05:01:35,MoonPlace - a place like r/place but on-chain ...,randomFrenchDeadbeat,j6cgo7d,Wow. Everything is sold XD\n\nThis is the firs...,8
4,2023-01-28 19:32:19,MoonPlace - a place like r/place but on-chain ...,Lord-Nagafen,j6aux29,Added Batman to the canvas! 🦇,7
5,2023-01-28 22:40:36,MoonPlace - a place like r/place but on-chain ...,Maxx3141,j6binb3,Who put that cone on me?! -_-,8
6,2023-01-29 04:34:48,MoonPlace - a place like r/place but on-chain ...,1078Garage,j6ceqbj,"I came for the tiles, I stayed for the crash c...",6
7,2023-01-29 04:50:01,MoonPlace - a place like r/place but on-chain ...,ukelgn,j6cfttp,Bought some plots from OS. Finally I learned t...,8
8,2023-01-27 19:35:12,MoonPlace - a place like r/place but on-chain ...,jwinterm,j66akqt,Working on upgrading server at the moment...,20
9,2023-01-26 18:47:13,MoonPlace - a place like r/place but on-chain ...,nanooverbtc,j6162fl,Can definitely see avatar enthusiasts minting ...,28


In [60]:
quer = """SELECT comments, upvote_score
FROM public."reddit_r_Cryptocurrency"
WHERE comments LIKE '%%bitcoin%%'
OR comments LIKE '%%ethereum%%'
ORDER BY upvote_score DESC
"""

pd.read_sql(quer, con=engine)

,comments,upvote_score
0,2017: BlackRock CEO Larry Fink calls bitcoin a...,99
1,Long story short: She was promising the next b...,97
2,I follow your posts because I think it's an im...,13
3,tldr; Bitcoin reached a fresh five-month high ...,10
4,How does it not make sense? Fiat is devaluing...,9
...,...,...
69,I'm on crypto and bitcoin since 2021. And noti...,0
70,"Finally 1M in Q12 2021!\n\nJokes aside, bitcoi...",0
71,https://dailyhodl.com/2022/08/09/cathie-wood-o...,0
72,"Short answer: No, you are pretty much guarante...",0
